## Packnet+pruning


In [2]:
import pandas as pd
import torch
import random
import seaborn as sns
import os
import pickle
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold

from torch.utils.data import DataLoader, Dataset, Subset
import torch.nn as nn
import torch.optim as optim

from transformers import BertTokenizer

import matplotlib.pyplot as plt
from perceiver import tokenize_data, CustomDataset, PerceiverBlock, Perceiver, CombinedModel

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LENGTH = 128

root_dir = '/home/youlee/perceiver/perceiver/model/'
loader_dir = '/home/youlee/perceiver/perceiver/loader/'

batch_size = 32

input_models = []
valid_loaders = []


for i in range(6):
    text_model = torch.load(root_dir + f'text_model_{i+1}.pkl')
    input_models.append(text_model)
    print(f"Text model {i+1}번 불러오기 완료.")

for i in range(6):
    img_model = torch.load(root_dir + f'image_model_{i+1}.pkl')
    input_models.append(img_model)
    print(f"Image model {i+1}번 불러오기 완료.")

for i in range(6):
    with open(loader_dir + f'text_val_loader_{i+1}.pkl', 'rb') as f:
        loaded_valid_dataset = pickle.load(f)
    valid_loaders.append(loaded_valid_dataset)
    print(f"Text val. loader {i}번 불러오기 완료.")

from torch.utils.data import DataLoader
for i in range(6):
    with open(loader_dir + f'image_val_loader_{i+1}.pkl', 'rb') as f:
        loaded_valid_dataset = pickle.load(f)
    valid_loader = DataLoader(loaded_valid_dataset, batch_size=batch_size, shuffle=False)
    valid_loaders.append(valid_loader)
    print(f"Image val. loader {i}번 불러오기 완료.")


class PackNet(nn.Module):
    def __init__(self, model):
        super(PackNet, self).__init__()
        self.model = model
        self.masks = {}
        self.current_task = None

    def set_task(self, task_id):
        self.current_task = task_id
        if task_id not in self.masks:
            self.masks[task_id] = {
                name: torch.ones_like(param, device=param.device)
                for name, param in self.model.named_parameters()
                if param.requires_grad
            }

    def prune(self, target_sparsity=0.2):
        if self.current_task is None:
            raise ValueError("Task must be set before pruning.")
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                mask = self.masks[self.current_task][name]
                threshold = torch.quantile(param.abs(), target_sparsity)
                mask[param.abs() < threshold] = 0
                self.masks[self.current_task][name] = mask

    def forward(self, input_ids, **kwargs):
       
        if self.current_task in self.masks:
            with torch.no_grad():
                for name, param in self.model.named_parameters():
                    if param.requires_grad:
                        param.data *= self.masks[self.current_task][name]
        return self.model(input_ids, **kwargs)


def eval_epoch(model, dataloader, criterion, device, is_text: bool):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            if is_text:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids)
            else:
                inputs, labels = batch
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

def gradual_pruning(packnet_model, model_type, model_index, criterion, device, 
                    start_sparsity, end_sparsity, pruning_steps, checkpoint_dir, valid_loaders):
    model_path = f"{checkpoint_dir}/{model_type}_model_{model_index+1}_pruned.pkl"  # .pt -> .pkl로 변경
    sparsity_increment = (end_sparsity - start_sparsity) / pruning_steps
    current_sparsity = start_sparsity
    
    if model_type == "text":
        test_loader = valid_loaders[model_index]     
        is_text = True
    else:
        test_loader = valid_loaders[model_index + 6] 
        is_text = False

    for step in range(pruning_steps):
        print(f"[{model_type.upper()} Model {model_index+1}] Pruning Step {step+1}/{pruning_steps}, sparsity={current_sparsity:.2f}")
        packnet_model.prune(target_sparsity=current_sparsity)
        current_sparsity += sparsity_increment

    # .pkl 형식으로 저장
    with open(model_path, 'wb') as f:
        pickle.dump({
            "model_state_dict": packnet_model.state_dict(),
            "masks": packnet_model.masks
        }, f)
    print(f"[{model_type.upper()} Model {model_index+1}] Pruned model saved: {model_path}")

    test_loss, test_acc = eval_epoch(packnet_model, test_loader, criterion, device, is_text=is_text)
    print(f"[{model_type.upper()} Model {model_index+1}] Final Test Accuracy: {test_acc:.4f}")
    print("---------")
    
    return packnet_model


if __name__ == "__main__":
    start_sparsity = 0.05
    end_sparsity = 0.2
    pruning_steps = 5
    checkpoint_dir = "/home/youlee/perceiver/perceiver/checkpoints_pruned"
    os.makedirs(checkpoint_dir, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = nn.CrossEntropyLoss()

    print("Starting gradual pruning process...")

    text_models = input_models[:6]
    image_models = input_models[6:]

    pruned_text_models = []
    for i, model in enumerate(text_models):
        packnet_model = PackNet(model)
        packnet_model.set_task(f"text_task_{i+1}")
        packnet_model.to(device)
        pruned_text_models.append(
            gradual_pruning(packnet_model, "text", i, criterion, device,
                            start_sparsity, end_sparsity, pruning_steps, checkpoint_dir, valid_loaders)
        )

    pruned_image_models = []
    for i, model in enumerate(image_models):
        packnet_model = PackNet(model)
        packnet_model.set_task(f"image_task_{i+1}")
        packnet_model.to(device)
        pruned_image_models.append(
            gradual_pruning(packnet_model, "image", i, criterion, device,
                            start_sparsity, end_sparsity, pruning_steps, checkpoint_dir, valid_loaders)
        )

    print("Gradual pruning process finished.")

/tmp/ipykernel_1401859/2962457805.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_model = torch.load(root_dir + f'text_model_{i+1}.pkl')
/tmp/ipykernel_1401859/296

Text model 1번 불러오기 완료.
Text model 2번 불러오기 완료.
Text model 3번 불러오기 완료.
Text model 4번 불러오기 완료.
Text model 5번 불러오기 완료.
Text model 6번 불러오기 완료.
Image model 1번 불러오기 완료.
Image model 2번 불러오기 완료.
Image model 3번 불러오기 완료.
Image model 4번 불러오기 완료.
Image model 5번 불러오기 완료.
Image model 6번 불러오기 완료.
Text val. loader 0번 불러오기 완료.
Text val. loader 1번 불러오기 완료.
Text val. loader 2번 불러오기 완료.
Text val. loader 3번 불러오기 완료.
Text val. loader 4번 불러오기 완료.
Text val. loader 5번 불러오기 완료.
Image val. loader 0번 불러오기 완료.
Image val. loader 1번 불러오기 완료.
Image val. loader 2번 불러오기 완료.
Image val. loader 3번 불러오기 완료.
Image val. loader 4번 불러오기 완료.
Image val. loader 5번 불러오기 완료.
Starting gradual pruning process...
[TEXT Model 1] Pruning Step 1/5, sparsity=0.05
[TEXT Model 1] Pruning Step 2/5, sparsity=0.08
[TEXT Model 1] Pruning Step 3/5, sparsity=0.11
[TEXT Model 1] Pruning Step 4/5, sparsity=0.14
[TEXT Model 1] Pruning Step 5/5, sparsity=0.17
[TEXT Model 1] Pruned model saved: /home/youlee/perceiver/perceiver/checkpoints_pruned/text_m